In [90]:
import torch
import os
import pandas as pd
import pickle
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

In [22]:
with open('small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [23]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
from modules.algorithms.nn.OSLPP_NN_UTILS import train_NN, predict_NN
from modules.loaders.osda import create_datasets_sub
from modules.algorithms.base.OSLPP import Params, do_l2_normalization, do_pca
from modules.loaders.common import set_seed

results = pd.DataFrame({'source': [],  'lr': [], 'seed': [], 'epochs': [], 'report': []})
used_src = set()
splitter = StratifiedKFold(n_splits=5)
for (source, target), (common, tgt_private) in config.items():
    if source in used_src:
        continue
    used_src.add(source)
    for epochs in [10, 50]:
        for lr in [1e-3, 1e-4]:
                set_seed(0)
                print(source, '->', target, 'lr=', lr, 'seed=', 0)
                params = Params(pca_dim=512, proj_dim=128, T=10, n_r=1200, n_r_ratio=None,
                          dataset='DomainNet_DCC', source=source, target=target,
                          num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                (feats_S, labels_S), (feats_T, labels_T) = create_datasets_sub(params.dataset,
                                                                               params.source,
                                                                               params.target,
                                                                               common,
                                                                               tgt_private)
                num_src_classes = params.num_common + params.num_src_priv

                # l2 normalization and pca
                feats_S, feats_T = do_l2_normalization(feats_S, feats_T)
                feats_S, feats_T = do_pca(feats_S, feats_T, params.pca_dim)
                feats_S, feats_T = do_l2_normalization(feats_S, feats_T)

                # initial
                feats_S, feats_T = torch.tensor(feats_S), torch.tensor(feats_T)
                labels_S, labels_T = torch.tensor(labels_S), torch.tensor(labels_T)

                for i, (train_idx, test_idx) in enumerate(splitter.split(feats_S, labels_S)):
                    feats_S_train, lbls_S_train = feats_S[train_idx], labels_S[train_idx]
                    feats_S_test, lbls_S_test = feats_S[test_idx], labels_S[test_idx]

                    model = train_NN(feats_S_train, lbls_S_train, epochs, params, balanced=True, lr=lr, initial=True)
                    feats_S_2, preds_S_2 = predict_NN(model, feats_S_test, lbls_S_test)
                    assert (feats_S_2 == feats_S_test).all()

                    confs, preds_labels = preds_S_2.max(dim=1)

                    np_preds = preds_labels.numpy()
                    np_labels = lbls_S_test.numpy()

                    report = classification_report(np_labels, np_preds, output_dict=True)

                    results = results.append({'source': source, 'lr': lr, 'seed': i, 'epochs': epochs, 'report': report}, ignore_index=True)

In [94]:
results.to_csv('results/dcc__source_only.csv', header=True, index=False)